# JRA Scraping Notebook

中央競馬（JRA）のデータをスクレイピングし、Google Drive上のデータセットに追加します。

In [ ]:
# 1. Google Driveのマウント
from google.colab import drive
drive.mount('/content/drive')

import os
import sys

# ★★★ 設定項目 ★★★
# scraperフォルダが存在するパス (Google Drive上のパス)
# 例: '/content/drive/MyDrive/dai-keiba'
PROJECT_PATH = '/content/drive/MyDrive/dai-keiba'

if not os.path.exists(PROJECT_PATH):
    print(f"Error: Path {PROJECT_PATH} does not exist. Please check your Drive structure.")
else:
    print(f"Project path found: {PROJECT_PATH}")
    os.chdir(PROJECT_PATH)
    sys.path.append(PROJECT_PATH)


In [ ]:
# 2. 必要なライブラリのインポート
try:
    import pandas as pd
    import requests
    import bs4
except ImportError:
    !pip install pandas requests beautifulsoup4
    import pandas as pd
    import requests
    import bs4

from datetime import datetime, date
from scraper.jra_scraper import scrape_jra_year, JRA_MONTH_PARAMS
import time

# ==========================================
# 🚨 2023年/2022年パラメータ不足の修正パッチ
# ==========================================
# パラメータマップを上書き更新
try:
    JRA_MONTH_PARAMS.update({
        "2023": { "01": "27", "02": "F5", "03": "C3", "04": "91", "05": "5F", "06": "2D", "07": "FB", "08": "C9", "09": "97", "10": "06", "11": "D4", "12": "A2" },
        "2022": { "01": "9B", "02": "69", "03": "37", "04": "05", "05": "D3", "06": "A1", "07": "6F", "08": "3D", "09": "0B", "10": "7A", "11": "48", "12": "16" }
    })
    print("✅ JRA 2023/2022 Parameters have been patched successfully.")
except NameError:
    print("⚠️ JRA_MONTH_PARAMS not found in scope. Ensure imports are correct.")


In [ ]:
# 3. スクレイピング実行関数の定義

def jra_scrape_execution(year_str, start_date=None, end_date=None):
    CSV_FILE_PATH = os.path.join(PROJECT_PATH, "database.csv")
    print(f"Using CSV Path: {CSV_FILE_PATH}")

    def save_chunk(df_chunk):
        if os.path.exists(CSV_FILE_PATH):
            try:
                # Read types as string to prevent auto-float for IDs
                existing_df = pd.read_csv(CSV_FILE_PATH, dtype={'race_id': str, 'horse_id': str})
                combined_df = pd.concat([existing_df, df_chunk], ignore_index=True)
            except Exception as e:
                print(f"Read Error: {e}, creating new.")
                combined_df = df_chunk
        else:
            combined_df = df_chunk

        # Deduplicate
        subset_cols = ['race_id', '馬名']
        subset_cols = [c for c in subset_cols if c in combined_df.columns]
        if subset_cols:
            combined_df.drop_duplicates(subset=subset_cols, keep='last', inplace=True)

        combined_df.to_csv(CSV_FILE_PATH, index=False, encoding="utf-8-sig")
        print(f"  [Saved] Total rows: {len(combined_df)} (+{len(df_chunk)} new)")

    print(f"Starting Scraping for {year_str} ({start_date} ~ {end_date})")

    # Load existing IDs to skip
    existing_ids = set()
    if os.path.exists(CSV_FILE_PATH):
        try:
             df_e = pd.read_csv(CSV_FILE_PATH, usecols=['race_id'], dtype={'race_id': str})
             existing_ids = set(df_e['race_id'].astype(str))
             print(f"  Loaded {len(existing_ids)} existing race IDs to skip.")
        except:
             pass

    scrape_jra_year(year_str, start_date=start_date, end_date=end_date, save_callback=save_chunk, existing_race_ids=existing_ids)


In [ ]:
# 4. 実行パラメータの設定と開始
# -----------------------------
TARGET_YEAR = "2024"
TARGET_MONTH = 1  # ★何月を取得するか指定 (Noneの場合は全期間、1〜12を指定)

import calendar
from datetime import date

START_DATE = None
END_DATE = None

if TARGET_MONTH:
    # 指定した月の1日〜末日を設定
    _, last_day = calendar.monthrange(int(TARGET_YEAR), int(TARGET_MONTH))
    START_DATE = date(int(TARGET_YEAR), int(TARGET_MONTH), 1)
    END_DATE = date(int(TARGET_YEAR), int(TARGET_MONTH), last_day)
    print(f"Targeting specific month: {START_DATE} to {END_DATE}")
else:
    # 自動判定ロジック (既存データの翌日から)
    CSV_FILE_PATH = os.path.join(PROJECT_PATH, "database.csv")
    if os.path.exists(CSV_FILE_PATH):
        try:
             df_exist = pd.read_csv(CSV_FILE_PATH)
             if '日付' in df_exist.columns and not df_exist.empty:
                 df_exist['date_obj'] = pd.to_datetime(df_exist['日付'], format='%Y年%m月%d日', errors='coerce')
                 last_date = df_exist['date_obj'].max()
                 if pd.notna(last_date):
                     # START_DATE = last_date.date() # 旧: 続きから
                     # 新: 欠落補完のために、強制的にその年の1月1日からスキャンする (existing_idsでスキップされる)
                     START_DATE = date(int(TARGET_YEAR), 1, 1)
                     print(f"既存データの最終日時: {last_date.date()} (欠落確認のため {START_DATE} からスキャンします)")
        except Exception as e:
            print(f"既存データ確認エラー: {e}")

print(f"Scraping Target: {TARGET_YEAR}, Start: {START_DATE}, End: {END_DATE}")
jra_scrape_execution(TARGET_YEAR, start_date=START_DATE, end_date=END_DATE)


In [ ]:
# 5. データ加工 (Feature Engineering) の実行
# ----------------------------------------
# スクレイピングした database.csv から学習用データを生成します
from ml.feature_engineering import calculate_features

INPUT_CSV = os.path.join(PROJECT_PATH, "database.csv")
OUTPUT_CSV = os.path.join(PROJECT_PATH, "processed_data.csv")

if os.path.exists(INPUT_CSV):
    print("Starting Feature Engineering...")
    calculate_features(INPUT_CSV, OUTPUT_CSV)
    print("Done!")
else:
    print("Error: database.csv not found.")
